In [145]:
from collections import OrderedDict
from joblib import dump, load

def encode_user_values(params):
    """
    Take in dictionary of values for the parameters and converts to format to be used by ML models
    - Hour 
    - Temp
    - Hum
    - Wind
    - Visb
    - Rainfall 
    - Snow
    - DWeek: Monday, Tuesday, Wednesday, Thursday, Friday, Saturday
    - Holiday 0 or 1
    - Fday: 0 or 1
    - Season: Autumn, Spring, Summer, Winter
    """
    # Initial checks
    if params["Fday"] not in [0,1]:
        raise ValueError("Fday value must be 0 or 1")
    
    if params["DWeek"] not in ["Monday","Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]:
        raise ValueError("Invalid day of week value")
    
    if params["Season"] not in ["Autumn", "Spring", "Summer", "Winter"]:
        raise ValueError("Season must be either Autumn, Spring, Summer, or Winter")
    
    """Order to follow internally
    ['Hour', 'Temp', 'Hum', 'Wind', 'Visb', 'Rainfall', 'Snow', 'Weekend',
       'Holiday', 'Fday', 'DWeek_Friday', 'DWeek_Monday', 'DWeek_Saturday',
       'DWeek_Sunday', 'DWeek_Thursday', 'DWeek_Tuesday', 'DWeek_Wednesday',
       'Seasons_Autumn', 'Seasons_Spring', 'Seasons_Summer', 'Seasons_Winter',
       'Count']"""
    
    ret_dict = OrderedDict()
    ret_dict["Hour"] = params["Hour"]
    ret_dict["Temp"] = params["Temp"]
    ret_dict["Hum"] = params["Hum"]
    ret_dict["Wind"] = params["Wind"]
    ret_dict["Visb"] = params["Visb"]
    ret_dict["Rainfall"] = params["Rainfall"]
    ret_dict["Snow"] = params["Snow"]
    ret_dict["Weekend"] = 1 * (params["DWeek"] in ["Saturday", "Sunday"])
    ret_dict["Holiday"] = params["Holiday"]
    ret_dict["Fday"] = params["Fday"]
    days = ["Friday","Monday", "Saturday", "Sunday", "Thursday", "Tuesday","Wedesday"]
    
    # Encode days
    for day in days: 
        if params["DWeek"] == day: 
            ret_dict["DWeek_" + day] = 1
        else:
            ret_dict["DWeek_" + day] = 0
    
    
    seasons = ["Autumn", "Spring", "Summer", "Winter"]
    for season in seasons: 
        if params["Season"] == season:
            ret_dict["Seasons_" + season] = 1
        else: 
            ret_dict["Seasons_" + season] = 0
            
    return list(ret_dict.values())

In [146]:
# CALL THIS FUNCTION WITH APPROPRIATE MODEL PATHS AND INPUT DATA TO GET PREDICTION
# also requires the function above ( encode user values)
# defaults to SVM 
def predict_bike_count(raw_input, MMS_PATH="./models/min_max_scaler.joblib", MODEL_PATH="./models/svm.joblib"):
    """
    raw_input is a dictionary format with the following values
    - Hour 
    - Temp
    - Hum
    - Wind
    - Visb
    - Rainfall 
    - Snow
    - DWeek: Monday, Tuesday, Wednesday, Thursday, Friday, Saturday
    - Holiday 0 or 1
    - Fday: 0 or 1
    - Season: Autumn, Spring, Summer, Winter
    
    MMS_PATH: is the relative path to the min max scaler file
    MODEL_PATH: the relative path to the appropriate model file
    """
    encoded_values = encode_user_values(raw_input)
    scaler = load(MMS_PATH) 
    scaled_values = scaler.transform([encoded_values])
    model = load(MODEL_PATH)
    pred = model.predict(scaled_values)
    ret_val = round(pred[0])
    return ret_val

In [147]:
test_input = {
    "Hour": 0,
    "Temp": -5.2,
    "Hum": 37,
    "Wind": 2.2,
    "Visb": 2000,
    "Rainfall": 0,
    "Snow": 0,
    "DWeek": "Friday",
    "Holiday": 1,
    "Season": "Winter",
    "Fday": 1
}
prediction = predict_bike_count(test_input) # Predicts on svm model by default
prediction

/Users/rekhathomas/opt/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


124

In [148]:
# using linear regression to predict ( has negative predictions sometimes unfortunately)
prediction = predict_bike_count(test_input, MODEL_PATH="./models/linear_regression.joblib")
prediction

/Users/rekhathomas/opt/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


-130